Other sources of data:
* http://eoddata.com/register.aspx
* barchart, databull, esignal, iqfeed, iris, mtpredictor, primate, prophet.net, qcollector, quotesplus, reuters, thinkorswim, tradeguider, tradestation, trading-tools.com, wealthlab pro (fidelity), yahoo finance

In [ ]:
import requests
import json

url = "https://search13f.com/api/cusip/screener"


headers = {
    "Content-Type":"application/x-www-form-urlencoded; charset=UTF-8",
"Cookie": {"sessionid":"gid7fvk5kd2r7ylzizz82qc37m515ull",
           "csrftoken":"TbAgC5RQRksPo0JO7QYEjntqk6eQxPcR",
           "_ga":"GA1.2.287848159.1484431024"},
"DNT":"1",
"Host":"search13f.com",
"Origin":"https://search13f.com",
"Referer":"https://search13f.com/screener",
}
    



In [ ]:
import time

results = []

for p in range(1,610):
    print(p)
    
    post_data = {
    "page":p,
    "per_page":20,
    "sort_by":"name",
    "query_parameters": '{"sliders":{"percentile_change_last_quarter_totalShares_relative":{"min":-100,"max":4033310.989},"percentile_current_totalValue":{"min":1,"max":331567521},"percentile_current_avgPrice":{"min":0.001,"max":1530000},"percentile_turnover":{"min":0,"max":100},"percentile_change_last_quarter_avgPrice_relative":{"min":-99.955,"max":2617233.333}},"has_symbol":false,"sector":"Any"}'}

    resp = requests.post(url, data = post_data, cookies=cookie)
    data = resp.json()
    results.extend(data["data"])
    time.sleep(1)


In [ ]:
import pandas

df = pandas.DataFrame(results)
cols_to_drop = [0,1,2,3,4,5,11]

df.drop(df.columns[cols_to_drop], axis=1, inplace=True)
df

In [ ]:
df.to_csv("cusips.csv", sep='\t', encoding='utf-8')

In [2]:
import pandas as pd

df = pd.read_csv("cusips.csv", sep='\t')
df.drop(df.columns[[0]], inplace=True,axis=1)


In [3]:
import requests 
import urllib

result = list()
base = "http://wikisynonyms.ipeirotis.com/api/"
for i, row in df[pd.isnull(df["stocksymbol"])==False].iterrows():
    cusip = row["cusip"]
    name = row["name"]
    stocksymbol = row["stocksymbol"]

    try:
        url = base + urllib.parse.quote(name)
        resp = requests.get(url)
    except:
        print("ERROR:", url)
        continue
    if (resp.status_code==204 or resp.status_code==404):
        continue
        
    data = resp.json()
    if len(data["terms"])>0:
        synonyms = [entry["term"] for entry in data["terms"]]
        for synonym in synonyms:
            entry = {"cusip": cusip,"name": name,"stocksymbol":stocksymbol, "synonym": synonym}
            result.append( entry )


NameError: name 'pandas' is not defined

In [7]:
df2 = pd.DataFrame(result)
df2.to_csv("cusips-synonyms.csv", sep='\t', encoding='utf-8')